<a href="https://colab.research.google.com/github/dhwanipatel14/Emotions-Detection-Model/blob/master/Live_ETD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from google.colab.patches import cv2_imshow
import cv2
import dlib
import numpy as np
from IPython.display import display, clear_output

# Download the shape predictor model
!wget "http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2"
!bzip2 -d shape_predictor_68_face_landmarks.dat.bz2

# Function to detect eyes in a frame
def detect_eyes(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)

    if faces:
        shape = predictor(gray, faces[0])
        left_eye = shape.parts()[36:42]
        right_eye = shape.parts()[42:48]
        return left_eye, right_eye
    else:
        return None, None

# Function to calculate Eye Aspect Ratio (EAR)
def calculate_ear(eye):
    eye = np.array([(point.x, point.y) for point in eye])
    A = np.linalg.norm(eye[1] - eye[5])
    B = np.linalg.norm(eye[2] - eye[4])
    C = np.linalg.norm(eye[0] - eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

# Load dlib face detector and facial landmarks predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("/content/shape_predictor_68_face_landmarks.dat")

# Initialize video capture from the camera
cap = cv2.VideoCapture(0)  # 0 corresponds to the default camera (you can change it if you have multiple cameras)

# Get video properties for the output video
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Initialize video writer for the output video
fourcc = cv2.VideoWriter_fourcc(*'XVID')
output_video = cv2.VideoWriter('/content/outputvideo.avi', fourcc, fps, (width, height))

# Initialize variables to record durations
duration_eyes_closed = 0
duration_looking_left = 0
duration_looking_right = 0
duration_looking_straight = 0

# Initialize variables for counting eye movement
count_left = 0
count_right = 0
count_straight = 0

while True:
    ret, frame = cap.read()

    if not ret:
        break

    left_eye, right_eye = detect_eyes(frame)

    if left_eye is not None and right_eye is not None:
        ear_left = calculate_ear(left_eye)
        ear_right = calculate_ear(right_eye)

        # Calculate the average EAR for both eyes
        avg_ear = (ear_left + ear_right) / 2.0

        # Set a threshold for distraction detection (you may need to adjust this)
        distraction_threshold = 0.2

        # Check if the person is distracted
        if avg_ear < distraction_threshold:
            cv2.putText(frame, "Eyes Closed", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 2)
            duration_eyes_closed += 1 / fps  # Increment the duration
            count_straight += 1

        else:
            # Check gaze direction
            horizontal_ratio = (left_eye[0].x + right_eye[3].x) / 2 / width
            if horizontal_ratio < 0.4:
                cv2.putText(frame, "Looking Left", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 2)
                duration_looking_left += 1 / fps  # Increment the duration
                count_left += 1
            elif horizontal_ratio > 0.6:
                cv2.putText(frame, "Looking Right", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 2)
                duration_looking_right += 1 / fps  # Increment the duration
                count_right += 1
            else:
                cv2.putText(frame, "Looking Straight", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 2)
                duration_looking_straight += 1 / fps  # Increment the duration

        # Draw contours around eyes
        for eye in [left_eye, right_eye]:
            for point in eye:
                x, y = point.x, point.y
                cv2.circle(frame, (x, y), 3, (0, 255, 0), -1)

    output_video.write(frame)  # Write the frame to the output video

    # Display the frame without modifying color
    cv2_imshow(frame)
    # Clear the previous output
    clear_output(wait=True)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object, video writer, and close all windows
cap.release()
output_video.release()
cv2.destroyAllWindows()

# Print the durations and most observed feature at the end
print(f"Duration taken looking right: {duration_looking_right} sec")
print(f"Duration taken closed eyes: {duration_eyes_closed} sec")
print(f"Duration taken looking left: {duration_looking_left} sec")
print(f"Duration taken looking straight: {duration_looking_straight} sec")

# Determine the most observed feature
max_duration = max(duration_looking_right, duration_eyes_closed, duration_looking_left, duration_looking_straight)
if max_duration == duration_looking_right:
    print("The most observed eye movement: Looking Right")
elif max_duration == duration_eyes_closed:
    print("The most observed eye movement: Eyes Closed")
elif max_duration == duration_looking_left:
    print("The most observed eye movement: Looking Left")
else:
    print("The most observed eye movement: Looking Straight")


--2023-12-07 22:56:24--  http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64040097 (61M)
Saving to: ‘shape_predictor_68_face_landmarks.dat.bz2’

shape_predictor_68_ 100%[===================>]  61.07M  15.2MB/s    in 8.4s    

2023-12-07 22:56:33 (7.24 MB/s) - ‘shape_predictor_68_face_landmarks.dat.bz2’ saved [64040097/64040097]

bzip2: Output file shape_predictor_68_face_landmarks.dat already exists.
Duration taken looking right: 0 sec
Duration taken closed eyes: 0 sec
Duration taken looking left: 0 sec
Duration taken looking straight: 0 sec
The most observed eye movement: Looking Right
